In [15]:
import threading
from queue import Queue
import pandas as pd
import numpy as np
from pylsl import StreamInlet, resolve_byprop

In [16]:
CQ_ch = ['CQ.OVERALL', 'CQ.singnal']
EQ_ch = ['EQ.OVERALL', 'EQ.SampleRateQuality']
ch = ['Cz', 'FCz', 'Fz', 'Fz', 'F3', 'FC1', 'C1', 'C3', 'FC3', 'FC5', 'C5','CP5', 'CP3', 
    'CP1', 'P1', 'P3', 'CPz', 'Pz', 'P4', 'P2',  'CP2', 'CP4', 'CP6', 'C6', 'FC6', 
    'FC4', 'C4', 'C2', 'FC2', 'F2','F4'] 

for i in range(len(ch)):
    CQ_ch.append('CQ.' + ch[i])
    EQ_ch.append('EQ.' + ch[i])
    
header = ['Timestamp'] + ch + CQ_ch + EQ_ch
    
df_all = pd.DataFrame(columns= header)

In [26]:
# データを格納するためのスレッドセーフキュー
eeg_queue = Queue()
cq_queue = Queue()
eq_queue = Queue()

# 各ストリームに対するワーカー関数を作成
def eeg_worker(eeg_inlet, eeg_queue):
    while True:
        eeg_sample, eeg_timestamp = eeg_inlet.pull_sample()
        eeg_queue.put((eeg_sample, eeg_timestamp))

def cq_worker(cq_inlet, cq_queue):
    while True:
        cq_sample, cq_timestamp = cq_inlet.pull_sample()
        cq_queue.put((cq_sample, cq_timestamp))

def eq_worker(eq_inlet, eq_queue):
    while True:
        eq_sample, eq_timestamp = eq_inlet.pull_sample()
        eq_queue.put((eq_sample, eq_timestamp))

def collect_data_multithreaded(duration, eeg_inlet, cq_inlet, eq_inlet, ch, CQ_ch, EQ_ch, df_all):
    # 各ストリームのスレッドを作成
    eeg_thread = threading.Thread(target=eeg_worker, args=(eeg_inlet, eeg_queue))
    cq_thread = threading.Thread(target=cq_worker, args=(cq_inlet, cq_queue))
    eq_thread = threading.Thread(target=eq_worker, args=(eq_inlet, eq_queue))

    # スレッドを開始
    eeg_thread.start()
    cq_thread.start()
    eq_thread.start()

    start_time = None
    while True:
        # キューからデータを取得。データがなければNaNを使用
        eeg_sample, eeg_timestamp = eeg_queue.get() if not eeg_queue.empty() else ([np.nan]*len(ch), np.nan)
        cq_sample, cq_timestamp = cq_queue.get() if not cq_queue.empty() else ([np.nan]*len(CQ_ch), np.nan)
        eq_sample, eq_timestamp = eq_queue.get() if not eq_queue.empty() else ([np.nan]*len(EQ_ch), np.nan)

        eeg_sample = eeg_sample[3:-2]
        eq_sample = eq_sample[2:]
        cq_sample = cq_sample[2:]
        
        print(len(eeg_sample))
        print(len(eq_sample))
        print(len(cq_sample))
        
        if start_time is None and not np.isnan(eeg_timestamp):
            start_time = eeg_timestamp  # 開始時間はEEGストリームに基づいています

        if start_time is not None:
            elapsed_time = eeg_timestamp - start_time  # 経過時間はEEGストリームに基づいています

        # 全サンプルを結合
        sample = eeg_sample + cq_sample + eq_sample
        df_all = save_data_to_csv(start_time, eeg_timestamp, sample, df_all)  # タイムスタンプはEEGストリームに基づいています

        if elapsed_time >= duration:
            break

    return df_all

def save_data_to_csv(start_time, timestamp, sample, df_all):
    # 開始と終了の3つの要素を削除し、タイムスタンプを0から始まるように修正
    timestamp -= start_time
    
    # DataFrameを作成し、データを追記していく
    # sampleはリスト形式でデータが渡される
    # マーカーもこのDataFrameに格納
    # タイムスタンプを先頭に追加
    df = pd.DataFrame([[timestamp] + sample], columns=header)
    df_all = pd.concat([df_all ,df])
    return df_all
    
#CSVファイルからデータを読み込む関数
def load_data_from_csv(filename):
    data = []
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)

    return data

# データをプロットする関数
def plot_data(data, channels):
    timestamps = [float(row['Timestamp']) for row in data]

    plt.figure(figsize=(15, 8))

    for channel in channels:
        values = [float(row[channel]) for row in data]
        plt.plot(timestamps, values, label=channel)

    plt.xlabel('Timestamp (s)')
    plt.ylabel('Sensor Value')
    plt.title('Brainwave Data')
    plt.legend()
    plt.show()
    
    
def lsl_to_local_datetime(lsl_timestamp):
    return datetime.fromtimestamp(lsl_timestamp)

In [27]:
# 収集するデータの期間（秒）
duration = 5

# Emotiv EPOCのストリームを探す
eeg_streams = resolve_byprop('type', 'EEG', timeout=5)
cq_streams = resolve_byprop('type', 'Contact-Quality', timeout=5)
eq_streams = resolve_byprop('type', 'EEG-Quality', timeout=5)

if not eeg_streams  or not eq_streams or not cq_streams:
    raise RuntimeError("Not all streams found. Make sure the EmotivPro LSL Connector is running and configured correctly.")

# ストリームを開く
eeg_inlet = StreamInlet(eeg_streams[0])
eq_inlet = StreamInlet(eq_streams[0])
cq_inlet = StreamInlet(cq_streams[0])

print("Connected to Emotiv EPOC FLEX streams")

# EEGを収集し、DataFrameに保存
df_all = collect_data_multithreaded(duration, eeg_inlet, cq_inlet, eq_inlet, ch, CQ_ch, EQ_ch, df_all)

# CSVファイル名を指定
filename = 'a.csv'

# DataFrameをCSVファイルに保存
df_all.to_csv(filename, index=False)

print("Collecting Data Completed")


Connected to Emotiv EPOC FLEX streams
26
31
31


TypeError: unsupported operand type(s) for -=: 'float' and 'NoneType'

In [25]:
df_all

,Timestamp,Cz,FCz,Fz,Fz,F3,FC1,C1,C3,FC3,...,EQ.CP4,EQ.CP6,EQ.C6,EQ.FC6,EQ.FC4,EQ.C4,EQ.C2,EQ.FC2,EQ.F2,EQ.F4


,Timestamp,Cz,FCz,Fz,Fz,F3,FC1,C1,C3,FC3,...,EQ.CP4,EQ.CP6,EQ.C6,EQ.FC6,EQ.FC4,EQ.C4,EQ.C2,EQ.FC2,EQ.F2,EQ.F4
